In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=81761ef36a590a783514e190d1afa8f126285e5a9d7241378ca534481311ceca
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master('local[1]')\
.appName('Amaze2.in')\
.getOrCreate()
from pyspark import SparkContext
sc=spark.sparkContext

In [ ]:
from pyspark.sql.functions import max,min,sum,count,col,when,like,lit,regexp_replace
from pyspark.sql.types import StructField,StructType,IntegerType,StringType

In [ ]:
df=spark.read.csv('sales_data.csv',header=True,inferSchema=True)
df.show()

+-------------+---------+------------+------------+---------------+
|TransactionID|ProductID|QuantitySold|PricePerUnit|TransactionDate|
+-------------+---------+------------+------------+---------------+
|           89|      176|           7|       77.24|     2021-01-04|
|           33|      182|           3|       34.05|     2021-01-05|
|           48|      174|           1|       53.91|     2021-01-08|
|           20|      188|           3|       40.51|     2021-01-14|
|           59|      105|           4|       52.25|     2021-01-21|
|           22|      137|           4|       25.39|     2021-01-23|
|           83|      175|           7|       23.83|     2021-01-25|
|           94|      199|           1|       51.68|     2021-01-27|
|           44|      109|           9|       97.32|     2021-01-30|
|           37|      129|           7|        96.8|     2021-01-31|
|           17|      139|           1|        77.5|     2021-02-08|
|           67|      101|           6|       39.

In [ ]:
df.createOrReplaceTempView("table")

result=spark.sql('''select ProductID,sum(QuantitySold) as Sales from table
                    group by ProductID order by sum(QuantitySold) desc''').show()
result=spark.sql('''select ProductID,sum(QuantitySold) as Sales from table
                    group by ProductID order by sum(QuantitySold) desc limit 1 ''').collect()
print(f"The ProductId with the highest sales is {result[0][0]} with sales of {result[0][1]}")


+---------+-----+
|ProductID|Sales|
+---------+-----+
|      165|   19|
|      158|   18|
|      100|   18|
|      147|   16|
|      146|   15|
|      109|   15|
|      153|   14|
|      123|   14|
|      114|   13|
|      179|   13|
|      142|   13|
|      168|   13|
|      132|   13|
|      164|   13|
|      157|   12|
|      188|   11|
|      175|   11|
|      120|   10|
|      131|   10|
|      135|   10|
+---------+-----+
only showing top 20 rows

The ProductId with the highest sales is 165 with sales of 19


In [ ]:
data=df.withColumn('TotalSales',((df.QuantitySold)*(df.PricePerUnit))).collect()

In [ ]:
df=spark.createDataFrame(data)
df.show()

+-------------+---------+------------+------------+---------------+------------------+
|TransactionID|ProductID|QuantitySold|PricePerUnit|TransactionDate|        TotalSales|
+-------------+---------+------------+------------+---------------+------------------+
|           89|      176|           7|       77.24|     2021-01-04|            540.68|
|           33|      182|           3|       34.05|     2021-01-05|102.14999999999999|
|           48|      174|           1|       53.91|     2021-01-08|             53.91|
|           20|      188|           3|       40.51|     2021-01-14|            121.53|
|           59|      105|           4|       52.25|     2021-01-21|             209.0|
|           22|      137|           4|       25.39|     2021-01-23|            101.56|
|           83|      175|           7|       23.83|     2021-01-25|            166.81|
|           94|      199|           1|       51.68|     2021-01-27|             51.68|
|           44|      109|           9|     

In [ ]:
data=df.withColumn('Month',when((df.TransactionDate).like("%-01-%"),lit("January")).when((df.TransactionDate).like("%-02-%"),lit("February")).\
              when((df.TransactionDate).like("%-03-%"),lit("March")).when((df.TransactionDate).like("%-04-%"),lit("April")).\
              when((df.TransactionDate).like("%-05-%"),lit("May")).when((df.TransactionDate).like("%-06-%"),lit("June")).\
              when((df.TransactionDate).like("%-07-%"),lit("July")).when((df.TransactionDate).like("%-08-%"),lit("August")).\
              when((df.TransactionDate).like("%-09-%"),lit("September")).when((df.TransactionDate).like("%-10-%"),lit("October")).\
              when((df.TransactionDate).like("%-11-%"),lit("November")).when((df.TransactionDate).like("%-12-%"),lit("December"))).collect()

In [ ]:
df=spark.createDataFrame(data)
df.show()

+-------------+---------+------------+------------+---------------+------------------+--------+
|TransactionID|ProductID|QuantitySold|PricePerUnit|TransactionDate|        TotalSales|   Month|
+-------------+---------+------------+------------+---------------+------------------+--------+
|           89|      176|           7|       77.24|     2021-01-04|            540.68| January|
|           33|      182|           3|       34.05|     2021-01-05|102.14999999999999| January|
|           48|      174|           1|       53.91|     2021-01-08|             53.91| January|
|           20|      188|           3|       40.51|     2021-01-14|            121.53| January|
|           59|      105|           4|       52.25|     2021-01-21|             209.0| January|
|           22|      137|           4|       25.39|     2021-01-23|            101.56| January|
|           83|      175|           7|       23.83|     2021-01-25|            166.81| January|
|           94|      199|           1|  

In [ ]:
df.createOrReplaceTempView("Myta")
df.show()
result=spark.sql("select Month,sum(TotalSales) as TotalSales from Myta group by Month order by TotalSales desc")
result.show()

+-------------+---------+------------+------------+---------------+------------------+--------+
|TransactionID|ProductID|QuantitySold|PricePerUnit|TransactionDate|        TotalSales|   Month|
+-------------+---------+------------+------------+---------------+------------------+--------+
|           89|      176|           7|       77.24|     2021-01-04|            540.68| January|
|           33|      182|           3|       34.05|     2021-01-05|102.14999999999999| January|
|           48|      174|           1|       53.91|     2021-01-08|             53.91| January|
|           20|      188|           3|       40.51|     2021-01-14|            121.53| January|
|           59|      105|           4|       52.25|     2021-01-21|             209.0| January|
|           22|      137|           4|       25.39|     2021-01-23|            101.56| January|
|           83|      175|           7|       23.83|     2021-01-25|            166.81| January|
|           94|      199|           1|  

In [ ]:
df4=spark.read.csv('sales_data 1.csv',header=True,inferSchema=True)
df4.show()


+---------+------------+-------------+
|ProductID|QuantitySold|StockQuantity|
+---------+------------+-------------+
|      100|          19|           29|
|      101|           4|           17|
|      102|           7|           20|
|      103|           4|           53|
|      104|           2|           21|
|      105|           3|           32|
|      107|           1|           32|
|      109|          14|           60|
|      110|           9|           42|
|      113|          13|           29|
|      114|           5|           39|
|      115|          11|           45|
|      116|          21|           43|
|      118|           8|           19|
|      119|           1|           49|
|      121|           2|           51|
|      122|           4|           37|
|      125|           1|           35|
|      126|           5|           32|
|      127|           9|           56|
+---------+------------+-------------+
only showing top 20 rows



In [ ]:
df4.createOrReplaceTempView('Table2')
result=spark.sql('''select count(ProductID) from Table2 where QuantitySold>StockQuantity''').collect()[0][0]
print(result)

0
